IMPORT LIBRARIES

In [1]:
import numpy
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import SGD, Adam
#from keras.wrappers.scikit_learn import KerasRegressor
#from sklearn.model_selection import cross_val_score
#from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler, MinMaxScaler
#from sklearn.pipeline import Pipeline
from sklearn.cross_validation import train_test_split


C:\Users\iliac\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
C:\Users\iliac\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


# IMPORT DATA

In [2]:
data = pd.read_excel("C:\\Users\\iliac\\Documents\\Machine Learning\\New Challenge\\SaqmeBot.xlsx")

SET VARIABLES

In [4]:
columns = ['District','Area','Rooms','Floor','Number_of_floors','Condition','Status','Project','Heat','Gas','Storage','Cellar','Balcony','Garage','gancxadeba','Price']

In [5]:
train_columns = ['District','Area','Rooms','Floor','Number_of_floors','Condition','Status','Project','Heat','Gas','Storage','Cellar','Balcony','Garage','gancxadeba']

In [6]:
amenities = ['Heat','Gas','Storage','Cellar','Balcony','Garage']

In [7]:
cat =  ['District','Condition','Status','Project'] 

In [8]:
num = ['Area','Rooms','Floor','Number_of_floors']

TRANSLATE COLUMN NAMES

In [9]:
data.columns = columns

## Preprocessing

Change Nan to 0 in Amenities

In [10]:
data[amenities] = data[amenities].fillna(0)

Drop other Nan-s

In [11]:
data.dropna(inplace=True)

Drop 'gancxadebebi'. We don't need that feature

In [12]:
data.drop('gancxadeba',axis=1,inplace=True)

Remove Fake data

In [13]:
data = data[(data['Area']>10) & (data['Area']<300)]  # Keep 'Area' from 10 to 300
data = data[(data['Floor']>1) & (data['Floor']<30)] # Keep 'Floor' from 1 to 30
data = data[(data['Number_of_floors']>1) & (data['Number_of_floors']<30)] # Keep 'Number_of_floors' from 1 to 30
data = data[(data['Price']>5000) & (data['Price']<2000000)] # Keep 'Price' from 5,000 to 2,000,000

Drop Duplicate Samples

In [14]:
data.drop_duplicates(inplace=True)

Create Dummies for Categorical Variables and drop first column for every feature

In [15]:
data = pd.get_dummies(data,columns=cat, drop_first=True)

MinMaxScale on Num Data

In [16]:
scaler = MinMaxScaler(feature_range=(0,1))
num = ['Area','Rooms','Floor','Number_of_floors']

In [17]:
scaled_features = scaler.fit_transform((data[num])) #Scaled features in 'num' columns

In [18]:
new = pd.DataFrame(scaled_features,columns=num) #Create 'new' DataFrame from scaled_features

In [19]:
data.reset_index(drop=True,inplace=True) #We should reset indexing in Data to merge it with 'new' Dataframe because their indexing are off (because of dropping rows)

In [20]:
data[num] = new[num] #Change 'num' columns in data with 'new' Dataframe

In [21]:
print(new.shape)
print(type(new))
print(data.shape)
print(type(data))
print(data[num].shape)

(37963, 4)
<class 'pandas.core.frame.DataFrame'>
(37963, 84)
<class 'pandas.core.frame.DataFrame'>
(37963, 4)


Create Label

In [22]:
y = data.pop('Price') #Price is our label

In [23]:
type(data)

pandas.core.frame.DataFrame

Split train test

In [24]:
X_train, X_test, y_train, y_test = train_test_split(data,y,test_size = 0.1,random_state=42)

In [25]:
print(X_train.shape)
print(type(X_train))

print(X_test.shape)
print(type(X_test))

print(y_train.shape)
print(type(y_train))

print(y_test.shape)
print(type(y_test))

(34166, 83)
<class 'pandas.core.frame.DataFrame'>
(3797, 83)
<class 'pandas.core.frame.DataFrame'>
(34166,)
<class 'pandas.core.series.Series'>
(3797,)
<class 'pandas.core.series.Series'>


In [32]:
X_train = X_train.values
y_train = y_train.values

In [33]:
X_train.shape

(34166, 83)

## Neural Network

In [45]:
model = Sequential()
model.add(Dense(300, input_shape=(83,),activation='relu'))
model.add(Dense(200, activation='relu'))
model.add(Dense(100, activation='relu'))
model.add(Dense(1))

In [46]:
model.compile(loss='mse',optimizer='rmsprop',metrics=['mse','mae','mape','cosine'])

In [36]:
type(X_train)

numpy.ndarray

In [47]:
model.fit(x=X_train,y=y_train,epochs=100,verbose=2,batch_size=10)
#model.fit(satesto_X_v,satesto_Y_v,epochs=20,verbose=2,batch_size=30)

Epoch 1/100
 - 9s - loss: 6457234940.5004 - mean_squared_error: 6457234940.5004 - mean_absolute_error: 42792.6959 - mean_absolute_percentage_error: 37.6420 - cosine_proximity: -9.9988e-01
Epoch 2/100
 - 8s - loss: 2934132962.2723 - mean_squared_error: 2934132962.2723 - mean_absolute_error: 23883.7091 - mean_absolute_percentage_error: 21.1253 - cosine_proximity: -1.0000e+00
Epoch 3/100
 - 8s - loss: 2823231199.4677 - mean_squared_error: 2823231199.4677 - mean_absolute_error: 22777.0533 - mean_absolute_percentage_error: 19.9312 - cosine_proximity: -1.0000e+00
Epoch 4/100
 - 8s - loss: 2784815414.0827 - mean_squared_error: 2784815414.0827 - mean_absolute_error: 22430.3277 - mean_absolute_percentage_error: 19.5988 - cosine_proximity: -1.0000e+00
Epoch 5/100
 - 8s - loss: 2749757362.2366 - mean_squared_error: 2749757362.2366 - mean_absolute_error: 22195.9725 - mean_absolute_percentage_error: 19.3488 - cosine_proximity: -1.0000e+00
Epoch 6/100
 - 8s - loss: 2740703108.7247 - mean_squared_err

Epoch 45/100
 - 9s - loss: 2303031266.5099 - mean_squared_error: 2303031266.5099 - mean_absolute_error: 19427.1291 - mean_absolute_percentage_error: 17.1683 - cosine_proximity: -1.0000e+00
Epoch 46/100
 - 8s - loss: 2304412501.0784 - mean_squared_error: 2304412501.0784 - mean_absolute_error: 19439.6587 - mean_absolute_percentage_error: 17.1959 - cosine_proximity: -1.0000e+00
Epoch 47/100
 - 8s - loss: 2284075469.4335 - mean_squared_error: 2284075469.4335 - mean_absolute_error: 19431.3109 - mean_absolute_percentage_error: 17.1198 - cosine_proximity: -1.0000e+00
Epoch 48/100
 - 9s - loss: 2283669851.7171 - mean_squared_error: 2283669851.7171 - mean_absolute_error: 19239.5808 - mean_absolute_percentage_error: 17.0034 - cosine_proximity: -1.0000e+00
Epoch 49/100
 - 9s - loss: 2279345191.5071 - mean_squared_error: 2279345191.5071 - mean_absolute_error: 19260.4260 - mean_absolute_percentage_error: 17.0106 - cosine_proximity: -1.0000e+00
Epoch 50/100
 - 9s - loss: 2253889880.8808 - mean_squar

Epoch 89/100
 - 8s - loss: 2047832833.8780 - mean_squared_error: 2047832833.8780 - mean_absolute_error: 18534.4878 - mean_absolute_percentage_error: 16.6501 - cosine_proximity: -1.0000e+00
Epoch 90/100
 - 8s - loss: 2042380893.5470 - mean_squared_error: 2042380893.5470 - mean_absolute_error: 18608.7445 - mean_absolute_percentage_error: 16.7040 - cosine_proximity: -1.0000e+00
Epoch 91/100
 - 9s - loss: 2050724752.8125 - mean_squared_error: 2050724752.8125 - mean_absolute_error: 18629.9177 - mean_absolute_percentage_error: 16.7622 - cosine_proximity: -1.0000e+00
Epoch 92/100
 - 8s - loss: 2032963909.1629 - mean_squared_error: 2032963909.1629 - mean_absolute_error: 18568.8855 - mean_absolute_percentage_error: 16.7125 - cosine_proximity: -1.0000e+00
Epoch 93/100
 - 8s - loss: 2034600974.2397 - mean_squared_error: 2034600974.2397 - mean_absolute_error: 18547.4673 - mean_absolute_percentage_error: 16.6698 - cosine_proximity: -1.0000e+00
Epoch 94/100
 - 8s - loss: 2019985013.1992 - mean_squar

## Evaluate

In [48]:
results = model.evaluate(X_test,y_test)

3797/3797 [==============================] - 0s 80us/step


In [49]:
results

[2135087146.6273375,
 2135087146.6273375,
 21699.59906412546,
 18.955640154887035,
 -0.9999999994976696]

## Save Model

In [408]:
model.save('trained_prediction.h5')
model.save_weights('trained_prediction_weights.h5')

## Testing

In [44]:
#Xnew = numpy.array([[6]])
# make a prediction
ynew = model.predict(gavtestot)
ynew

array([[62642.01]], dtype=float32)